In [1]:
import pandas as pd
dataset=pd.read_csv("Social_Network_Ads.csv")
dataset

,User ID,Gender,Age,EstimatedSalary,Purchased
0,15624510,Male,19,19000,0
1,15810944,Male,35,20000,0
2,15668575,Female,26,43000,0
3,15603246,Female,27,57000,0
4,15804002,Male,19,76000,0
...,...,...,...,...,...
395,15691863,Female,46,41000,1
396,15706071,Male,51,23000,1
397,15654296,Female,50,20000,1
398,15755018,Male,36,33000,0


In [4]:
dataset=dataset.drop("User ID",axis=1)

In [5]:
independent=dataset.drop("Purchased",axis=1)
dependent=dataset["Purchased"]

In [7]:
dataset["Purchased"].value_counts()

Purchased
0    257
1    143
Name: count, dtype: int64

In [8]:
from sklearn.model_selection import train_test_split
Xtrain,Xtest,Ytrain,Ytest=train_test_split(independent,dependent,random_state=0,test_size=0.3)

In [17]:
from sklearn.preprocessing import StandardScaler,OneHotEncoder
from sklearn.compose import ColumnTransformer
Preprocess=ColumnTransformer(transformers=[
    ('num',StandardScaler(),['Age','EstimatedSalary']),
    ('cat',OneHotEncoder(drop="first"),['Gender'])
])

In [23]:
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
pipe=Pipeline(steps=[
    ("pre",Preprocess),
    ("model",LogisticRegression())
])

In [40]:
from sklearn.model_selection import GridSearchCV
#Cfloat, default=1.0
#penalty{‘l1’, ‘l2’, ‘elasticnet’, None}, default=’l2’
#solver{‘lbfgs’, ‘liblinear’, ‘newton-cg’, ‘newton-cholesky’, ‘sag’, ‘saga’}, default=’lbfgs’
#'l1' works only with liblinear/saga; 'l2' works with all solvers; lbfgs, newton-cg, newton-cholesky, sag only support l2/none.
#param_grid={"model__C":[0.001, 0.01, 0.1, 1.0, 10.0, 100.0]}
param_grid = [
    {"model__penalty": ["l1"], "model__solver": ["liblinear", "saga"], "model__C": [0.001, 0.01, 0.1, 1.0, 10.0, 100.0]},
    {"model__penalty": ["l2"], "model__solver": ["newton-cg", "lbfgs", "liblinear", "sag", "saga", "newton-cholesky"], "model__C": [0.001, 0.01, 0.1, 1.0, 10.0, 100.0]},
]
#,'model__penalty': ['l1', 'l2'],"model__solver":['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga','newton-cholesky']}
grid=GridSearchCV(pipe,param_grid,n_jobs=-1,verbose=3)
grid.fit(Xtrain,Ytrain)

Fitting 5 folds for each of 48 candidates, totalling 240 fits


GridSearchCV(estimator=Pipeline(steps=[('pre',
                                        ColumnTransformer(transformers=[('num',
                                                                         StandardScaler(),
                                                                         ['Age',
                                                                          'EstimatedSalary']),
                                                                        ('cat',
                                                                         OneHotEncoder(drop='first'),
                                                                         ['Gender'])])),
                                       ('model', LogisticRegression())]),
             n_jobs=-1,
             param_grid=[{'model__C': [0.001, 0.01, 0.1, 1.0, 10.0, 100.0],
                          'model__penalty': ['l1'],
                          'model__solver': ['liblinear', 'saga']},
                         {'model__C': [0.001, 0.01, 0.1, 1.0, 10.0, 100.0],
                          'model__penalty': ['l2'],
                          'model__solver': ['newton-cg', 'lbfgs', 'liblinear',
                                            'sag', 'saga',
                                            'newton-cholesky']}],
             verbose=3)

In [41]:
grid.best_params_

{'model__C': 0.1, 'model__penalty': 'l1', 'model__solver': 'liblinear'}

In [42]:
Ypred=grid.predict(Xtest)
from sklearn.metrics import confusion_matrix
cm=confusion_matrix(Ytest,Ypred)
print(cm)

[[74  5]
 [ 7 34]]


In [43]:
from sklearn.metrics import classification_report
clf_report=classification_report(Ytest,Ypred)
print(clf_report)

              precision    recall  f1-score   support

           0       0.91      0.94      0.93        79
           1       0.87      0.83      0.85        41

    accuracy                           0.90       120
   macro avg       0.89      0.88      0.89       120
weighted avg       0.90      0.90      0.90       120

